In [1]:
import os

In [2]:
# check the folder
%pwd

'e:\\Chicken_DIsease\\Chicken_Disease_Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Chicken_DIsease\\Chicken_Disease_Classification'

In [5]:
# Entity :- Return Type of a function like ConfigBox
# Paste in E:\Chicken_DIsease\Chicken_Disease_Classification\src\Chicken_Disease_Classification\entity\config_entity.py

from dataclasses import dataclass
from pathlib import Path

# Config.yaml
@dataclass(frozen=True)
class DataIngestionConfig:  # It is not an actual class , but a data class
    root_dir:Path
    source_URL:str
    local_data_file:Path
    unzip_dir:Path


In [6]:
from Chicken_Disease_Classification.constrants import * # Import Everything
from Chicken_Disease_Classification.utils.common import read_yaml,create_directories 

In [7]:
# Config Manager
# Paste in Configuration (src\Chicken_Disease_Classification\config\configuration.py)

class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,  # Return Box Type  # Ctrl+click to check the file path
            params_filepath=PARAMS_FILE_PATH):

            self.config=read_yaml(config_filepath)
            self.params=read_yaml(params_filepath)

            # From common.py
            create_directories([self.config.artifacts_root]) # I can call using the key name using Box Type

    def get_data_ingestion_config(self) -> DataIngestionConfig: # calling Data ingest config
        config=self.config.data_ingestion  # Storing the config

        create_directories([config.root_dir]) # Check config 

        # Define the custom return type of the function, check config, storing it
        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir    
        )

        return data_ingestion_config
    

In [8]:
# If i call the function get data ingestion config, then i can access all the variable
# Update the components
# src\Chicken_Disease_Classification\components\data_ingestion.py

import os
import urllib.request as request  # Download the data set from url
import zipfile # Unzip the data
from Chicken_Disease_Classification import logger # Logging
from Chicken_Disease_Classification.utils.common import get_size # Get datasize

In [9]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig): # we will get get data ingestion config
        self.config=config

    # Download File Method
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):  # Check the directory,if not exists
            filename,headers=request.urlretrieve(           # UrlRetrive:-Retrieve a URL into a temporary location on disk.
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with the following info: \n{headers}") # Header=url
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    # Extract the zip file
    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path) # Extract all file here (unzip_path)
            

    

In [10]:
## Create The pipeline
# src\Chicken_Disease_Classification\pipeline\stage_01_data_ingestion.py

try:
    config=ConfigurationManager()
    data_ingestin_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestin_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-27 14:58:52,372: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-27 14:58:52,386: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-27 14:58:52,386: INFO: common: created directory at: artifacts]
[2024-08-27 14:58:52,386: INFO: common: created directory at: artifacts/data_ingestion]
[2024-08-27 14:59:34,544: INFO: 2693401752: artifacts/data_ingestion/data.zip download! with the following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: FDC5:9F087:AA67C:DBDCD:66CD9C56
Accept-Ranges: bytes
Date: Tue, 27 Aug 2024 09:28:55 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10229-MAA
X-Cache:

In [ ]:
# down load the ip along with unzipped the data in artifacts folder